In [1]:
graph = {
    'A': {'OR': ['B'], 'AND': ['C', 'D']},
    'B': {'OR': ['E', 'F']},
    'C': {'OR': ['G'], 'AND': ['H', 'I']},
    'D': {'OR': ['J']},
    'E': {},
    'F': {},
    'G': {},
    'H': {},
    'I': {},
    'J': {}
}
h = {'A':-1, 'B': 4, 'C': 2, 'D': 3, 'E': 6, 'F': 8, 'G': 2, 'H': 0, 'I':0, 'J':0}

In [2]:
def AO_star(g, h, root, goal):
    open = [root]
    closed = []
    while goal not in closed:
        print(open, closed)
        x = open.pop(0)
        closed.append(x)
        sub_graph = g[x]
        f_n_or = []
        f_n_and = 9999
        chk1, chk2 = 0, 0
        if 'OR' in sub_graph.keys():
            chk1 = 1
            or_part = sub_graph['OR']
            for i in or_part:
                f_n_or.append(1+h[i])
            print(or_part, f_n_or)
        if 'AND' in sub_graph.keys():
            chk2 = 1
            and_part = sub_graph['AND']
            f_n_and = 0
            for i in and_part:
                f_n_and += 1+h[i]
            print(and_part, f_n_and)
        if chk1==1 or chk2==1:
            min_f_n_or = min(min(f_n_or), 9999)

            if min_f_n_or<=f_n_and:
                open.append(or_part[f_n_or.index(min_f_n_or)])
            else:
                open.extend(and_part)
            m = min(f_n_and, min_f_n_or)
            print(x, m)
            if x!=root and h[x]!=m:
                h[x]=m
                for i in g.keys():
                    a = []
                    for j in g[i].values():
                        a+=j
                    if x in a:
                        open = [i]
                        closed=[]
                        break
    print(open, closed)
    return closed


path = AO_star(graph, h, 'A', 'J')
print("path = ",path)

['A'] []
['B'] [5]
['C', 'D'] 7
A 5
['B'] ['A']
['E', 'F'] [7, 9]
B 7
['A'] []
['B'] [8]
['C', 'D'] 7
A 7
['C', 'D'] ['A']
['G'] [3]
['H', 'I'] 2
C 2
['D', 'H', 'I'] ['A', 'C']
['J'] [1]
D 1
['A'] []
['B'] [8]
['C', 'D'] 5
A 5
['C', 'D'] ['A']
['G'] [3]
['H', 'I'] 2
C 2
['D', 'H', 'I'] ['A', 'C']
['J'] [1]
D 1
['H', 'I', 'J'] ['A', 'C', 'D']
['I', 'J'] ['A', 'C', 'D', 'H']
['J'] ['A', 'C', 'D', 'H', 'I']
[] ['A', 'C', 'D', 'H', 'I', 'J']
path =  ['A', 'C', 'D', 'H', 'I', 'J']


In [3]:
def Cost(H, condition, weight = 1):
	cost = {}
	if 'AND' in condition:
		AND_nodes = condition['AND']
		Path_A = ' AND '.join(AND_nodes)
		PathA = sum(H[node]+weight for node in AND_nodes)
		cost[Path_A] = PathA

	if 'OR' in condition:
		OR_nodes = condition['OR']
		Path_B =' OR '.join(OR_nodes)
		PathB = min(H[node]+weight for node in OR_nodes)
		cost[Path_B] = PathB
	return cost

def update_cost(H, Conditions, weight=1):
	Main_nodes = list(Conditions.keys())
	Main_nodes.reverse()
	least_cost= {}
	for key in Main_nodes:
		condition = Conditions[key]
		print(key,':', Conditions[key],'>>>', Cost(H, condition, weight))
		c = Cost(H, condition, weight)
		H[key] = min(c.values())
		least_cost[key] = Cost(H, condition, weight)
	return least_cost

def shortest_path(Start,Updated_cost, H):
	Path = Start
	if Start in Updated_cost.keys():
		Min_cost = min(Updated_cost[Start].values())
		key = list(Updated_cost[Start].keys())
		values = list(Updated_cost[Start].values())
		Index = values.index(Min_cost)

		Next = key[Index].split()
		if len(Next) == 1:

			Start =Next[0]
			Path += '<--' +shortest_path(Start, Updated_cost, H)
		else:
			Path +='<--('+key[Index]+') '

			Start = Next[0]
			Path += '[' +shortest_path(Start, Updated_cost, H) + ' + '

			Start = Next[-1]
			Path += shortest_path(Start, Updated_cost, H) + ']'

	return Path



H = {'A': -1, 'B': 5, 'C': 2, 'D': 4, 'E': 7, 'F': 9, 'G': 3, 'H': 0, 'I':0, 'J':0}

Conditions = {
'A': {'OR': ['B'], 'AND': ['C', 'D']},
'B': {'OR': ['E', 'F']},
'C': {'OR': ['G'], 'AND': ['H', 'I']},
'D': {'OR': ['J']}
}

weight = 1

print('Updated Cost :')
Updated_cost = update_cost(H, Conditions, weight=1)
print('*'*75)
print('Shortest Path :\n',shortest_path('A', Updated_cost,H))

Updated Cost :
D : {'OR': ['J']} >>> {'J': 1}
C : {'OR': ['G'], 'AND': ['H', 'I']} >>> {'H AND I': 2, 'G': 4}
B : {'OR': ['E', 'F']} >>> {'E OR F': 8}
A : {'OR': ['B'], 'AND': ['C', 'D']} >>> {'C AND D': 5, 'B': 9}
***************************************************************************
Shortest Path :
 A<--(C AND D) [C<--(H AND I) [H + I] + D<--J]
